## Configuração de conexão Pyspark

In [1]:
import time
from datetime import timedelta

from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv
load_dotenv(os.path.join('config', '.env'))

# Configurar SparkSession
spark = SparkSession.builder \
    .appName("PostgreSQL to Spark") \
    .config("spark.jars", "/caminho/para/postgresql-<versao>.jar") \
    .getOrCreate()

# Carregar variáveis de ambiente do arquivo .env

# Configurações do banco de dados
db_config = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

# URL de conexão JDBC
jdbc_url = f"jdbc:postgresql://{db_config['host']}:{db_config['port']}/{db_config['dbname']}"

# Propriedades de conexão
connection_properties = {
    "user": db_config['user'],
    "password": db_config['password'],
    "driver": "org.postgresql.Driver"
}

In [2]:
start_time_geral = time.time()

## Configuração de conexão ao Spotify

In [3]:
import os
from dotenv import load_dotenv 
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import src.spotify_data as spotify_data

load_dotenv(os.path.join('config', '.env'))
# Obter as credenciais do ambiente 
client_id = os.getenv('CLIENT_ID') 
client_secret = os.getenv('CLIENT_SECRET')

# Configuração da autenticação
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Coleta de dados: Banco de dados

In [4]:
start_time = time.time()

In [ ]:
df = spark.read.jdbc(url=jdbc_url, table="all_tracks_registry", properties=connection_properties)
df.show(4)


In [ ]:
df.count()

In [ ]:
df.printSchema()

In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
# Converter o tempo decorrido para o formato HH:MM:SS 
print(f"Tempo de processamento: {str(timedelta(seconds=int(elapsed_time)))}")

## Coleta dados do Spotify e cria novos bancos

In [9]:
start_time = time.time()

In [10]:
# Funções de banco de dados Tracks e Artistics
import src.db_table_track as db_table_track
import src.db_table_artistcs as db_table_artistcs

In [ ]:
# Selecionar e coletar os dados distintos 
distinct_uris = df.select('spotify_track_uri').distinct().collect() 
# Transformar em lista de valores simples 
uris_list = [row['spotify_track_uri'] for row in distinct_uris]
print(f'Quantidade de faixas: {len(uris_list)}')

In [ ]:
n = 0
for track_uri in uris_list:
    # Coleta de dados da faixa no spotify 
    dados_faixa = spotify_data.obter_dados_faixa(sp = sp, track_uri = track_uri)
    # Inseri dados da faixa no banco
    db_table_track.insert_track_data(dados_faixa)

    # Coleta de dados do Artista no Spotify
    artist_id = dados_faixa['main_artist_URI']
    dados_artista = spotify_data.obter_dados_artista(sp=sp, artist_uri = artist_id)
    # Inseri dados do Artista no Banco
    db_table_artistcs.insert_artist_data(dados_artista)
    print(f'Faixa número:{n}')
    n = n + 1
    

In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
# Converter o tempo decorrido para o formato HH:MM:SS 
print(f"Tempo de processamento: {str(timedelta(seconds=int(elapsed_time)))}")


In [ ]:
end_time_geral = time.time()

In [ ]:
# import pyspark.sql.functions as F
# df = df.withColumn('ano-mes-dia', F.to_date(F.col("ts")))
# df = df.withColumn("ano-mes", F.concat_ws("-", F.year(df["ts"]), F.month(df["ts"])))
# df = df.withColumn("ano_mes_str", F.date_format(df["ts"], "yyyy-MM")) 
# df = df.withColumn("ano_mes_date", F.to_date(df["ano_mes_str"], "yyyy-MM"))
# df.show(5, truncate=False)